In [1]:
import csv
import os
dir=os.listdir("./")
print(dir)

with open('./Training Dataset_v2/訓練資料集/sentence_retrieval_val.csv', newline='') as csvfile:

    rows = csv.reader(csvfile)
    count=0
    for row in rows:
        pass

['Public Test Data and Submission Template', '.git', 'indexes', 'Training Dataset_v2', 'aicup_trainmodel.py', '.gitignore', 'opencsv_test.py', 'test.ipynb', '資料集處理.ipynb', 'requirements.txt', 'model_train.ipynb']


In [2]:
!pip install transformers datasets accelerate

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!nvidia-smi

Sun May 14 14:42:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
|  0%   43C    P8    17W / 280W |      1MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:02:00.0 Off |                  N/A |
|  0%   

# 抓dataset

In [4]:
#from datasets import list_datasets, load_dataset
from pprint import pprint

In [5]:
"""from datasets import load_dataset

dataset = load_dataset("glue", "mrpc")"""

'from datasets import load_dataset\n\ndataset = load_dataset("glue", "mrpc")'

## dataset裡分成三個 train(訓練用) validation(每次訓縣完測試訓練情形用) test(最終算分用)

In [6]:
import json
from pprint import pprint


## 把每一筆資料集的每一項拆開

In [7]:
import csv
def read_data(dataset):
    with open(dataset,newline='')as csvfile:
        rows = csv.reader(csvfile)
        count=0
        sentence1=[]
        sentence2=[]
        label=[]
        for row in rows:
            if count==0:
                count+=1
                continue
            sentence1.append(row[0])
            sentence2.append(row[1])
            label.append(row[2])
            count+=1
    #open file
    
    
    return sentence1,sentence2,label

In [8]:
train_sen1,train_sen2,trainlabel=read_data("./Training Dataset_v2/訓練資料集/sentence_retrieval_train.csv")
#trainlabel=train_data['label']
print(len(train_sen1))
print(train_sen1[0])

7701
天衛三軌道在天王星內部的磁層，以《 仲夏夜之夢 》作者緹坦妮雅命名。


In [9]:
test_sen1,test_sen2,testlabel=read_data("./Training Dataset_v2/訓練資料集/sentence_retrieval_test.csv")
print(len(test_sen1))
print(test_sen1[0])

856
元智大學創辦人的兒子是現任班長。


In [10]:
eval_sen1,eval_sen2,evallabel=read_data('./Training Dataset_v2/訓練資料集/sentence_retrieval_val.csv')
#evalabel=eval_data['label']
print(len(eval_sen1))
print(eval_sen1[0])

951
中國政法大學被中國共產黨前主要領導人之一的李嵐清譽爲 “ 中國法學教育的最高學府 ” ，在世界法律院校內享有盛名。


In [11]:
print('train 資料集數量= ',len(train_sen1))
print('eval 資料集數量= ',len(eval_sen1))
print('test 資料集數量= ',len(test_sen1))

train 資料集數量=  7701
eval 資料集數量=  951
test 資料集數量=  856


# 選擇和使用tokenizer，具體要使用哪個tokenizer可以去huggingface官網找

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [12]:
pip install transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#from transformers import AutoTokenizer,BertTokenizerFast, BertTokenizer, AdamW, BertForQuestionAnswering
#tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")懶人用法
#tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
#tokenizer_fast = BertTokenizerFast.from_pretrained("bert-base-uncased")
#from transformers import BertTokenizer, BertForSequenceClassification

#tokenizer = BertTokenizer.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

ModuleNotFoundError: No module named 'transformers'

In [ ]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [ ]:
train_encodings = tokenizer(train_sen1, train_sen2, truncation=True, padding=True)
print(train_encodings)
eval_encodings = tokenizer(eval_sen1, eval_sen2, truncation=True, padding=True)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:


print(tokenizer.decode(train_encodings['input_ids'][0]))
print(train_encodings['input_ids'][0])
print("token_type_ids\n", train_encodings['token_type_ids'][0])
print("attention_mask\n", train_encodings['attention_mask'][0])

print(len(train_encodings['input_ids'][0]))
print(len(train_encodings['token_type_ids'][0]))

[CLS] [UNK] accused his brother, whom he called " the witness ", of deliberately distorting his evidence. [SEP] [UNK] to him as only " the witness ", [UNK] accused his brother of deliberately distorting his evidence. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[101, 100, 9226, 11609, 8303, 10163, 8575, 11759, 117, 9372, 8175, 9245, 9816, 8303, 107, 8174, 8541, 8165, 10077, 107, 117, 8205, 10843, 8169, 8956, 9690, 8436, 9796, 8415, 10143, 8221, 10163, 12311, 9552, 9056, 119, 102, 100, 8228, 8913, 8175, 8975, 10110, 107, 8174, 8541, 8165, 10077, 107, 117, 100, 9226, 11609, 8303, 10163, 8575, 11759, 8205, 10843, 8169, 8956, 9690, 8436, 9796, 8415, 1

## 加入label(答案)

In [ ]:
def add_targets(encodings,label):
    encodings.update({'label':label})
add_targets(train_encodings,trainlabel)
add_targets(eval_encodings,evallabel)


In [ ]:
print(train_encodings.keys())
print(tokenizer.decode(train_encodings['label']))

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'label'])
[unused1] [PAD] [unused1] [PAD] [unused1] [unused1] [PAD] [unused1] [PAD] [PAD] [PAD] [PAD] [unused1] [unused1] [PAD] [PAD] [PAD] [unused1] [PAD] [unused1] [unused1] [unused1] [PAD] [unused1] [unused1] [unused1] [PAD] [PAD] [unused1] [unused1] [unused1] [PAD] [unused1] [unused1] [unused1] [unused1] [PAD] [unused1] [PAD] [unused1] [unused1] [PAD] [unused1] [unused1] [unused1] [unused1] [unused1] [unused1] [unused1] [PAD] [unused1] [unused1] [PAD] [PAD] [PAD] [unused1] [PAD] [PAD] [unused1] [unused1] [unused1] [unused1] [PAD] [unused1] [unused1] [unused1] [unused1] [unused1] [unused1] [unused1] [PAD] [unused1] [PAD] [unused1] [unused1] [unused1] [unused1] [unused1] [PAD] [PAD] [unused1] [PAD] [PAD] [unused1] [PAD] [PAD] [unused1] [unused1] [PAD] [unused1] [unused1] [unused1] [PAD] [unused1] [unused1] [unused1] [unused1] [unused1] [unused1] [PAD] [PAD] [unused1] [PAD] [unused1] [PAD] [unused1] [unused1] [unused1] [unuse

# 定義dataset 並轉換成tensor格式


In [ ]:

from torch.utils import data
import torch

class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings

  def __getitem__(self, idx):
    return {key: torch.tensor(eval[idx]) for key, eval in self.encodings.items()}

  def __len__(self):
    return len(self.encodings.input_ids)

In [ ]:
train_dataset = Dataset(train_encodings)
eval_dataset = Dataset(eval_encodings)

In [ ]:
train_dataset[0]

{'input_ids': tensor([  101,   100,  9226, 11609,  8303, 10163,  8575, 11759,   117,  9372,
          8175,  9245,  9816,  8303,   107,  8174,  8541,  8165, 10077,   107,
           117,  8205, 10843,  8169,  8956,  9690,  8436,  9796,  8415, 10143,
          8221, 10163, 12311,  9552,  9056,   119,   102,   100,  8228,  8913,
          8175,  8975, 10110,   107,  8174,  8541,  8165, 10077,   107,   117,
           100,  9226, 11609,  8303, 10163,  8575, 11759,  8205, 10843,  8169,
          8956,  9690,  8436,  9796,  8415, 10143,  8221, 10163, 12311,  9552,
          9056,   119,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

# 載入模型架構

In [ ]:
from transformers import BertConfig, BertForSequenceClassification
config = BertConfig.from_pretrained('bert-base-chinese', num_labels=2)  #num_labels 設定類別數
model = BertForSequenceClassification.from_pretrained("bert-base-chinese",config=config)    

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

## 該來訓練模型囉

In [ ]:
import logging
import datasets
from datasets import load_dataset, load_metric
from torch.utils.data import DataLoader
from tqdm.auto import tqdm, trange
import math

import transformers
from accelerate import Accelerator
from transformers import (
    AdamW,
    AutoConfig,
    default_data_collator,
    get_scheduler
)

train_batch_size = 12      # 設定 training batch size 
eval_batch_size = 12      # 設定 eval batch size
num_train_epochs = 1      # 設定 epoch 

In [ ]:
data_collator = default_data_collator
train_dataloader = DataLoader(train_dataset, shuffle=True, collate_fn=data_collator, batch_size=train_batch_size)
eval_dataloader = DataLoader(eval_dataset, collate_fn=data_collator, batch_size=eval_batch_size)

In [ ]:
learning_rate=3e-5          # 設定 learning_rate
gradient_accumulation_steps = 1   # 設定 幾步後進行反向傳播

no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },                                
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

# Scheduler and math around the number of training steps.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
max_train_steps = num_train_epochs * num_update_steps_per_epoch
print('max_train_steps', max_train_steps)

# scheduler
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=max_train_steps,
)

max_train_steps 306


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:

# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator = Accelerator()

# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

eval_dataloader
metric = load_metric("accuracy")

真正開始訓練

In [ ]:
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(accelerator.state)
output_dir = '/content'  # your folder
 

total_batch_size = train_batch_size * accelerator.num_processes * gradient_accumulation_steps

logger.info("***** Running training *****")
logger.info(f"  Num examples = {len(train_dataset)}")
logger.info(f"  Num Epochs = {num_train_epochs}")
logger.info(f"  Instantaneous batch size per device = {train_batch_size}")
logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f"  Gradient Accumulation steps = {gradient_accumulation_steps}")
logger.info(f"  Total optimization steps = {max_train_steps}")


completed_steps = 0
best_epoch = {"epoch": 0, "acc": 0 }

for epoch in trange(num_train_epochs, desc="Epoch"):#trange是print進度條的方式
  model.train()
  for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
    outputs = model(**batch)
    #loss = outputs
    loss = outputs.loss
    #loss = loss / gradient_accumulation_steps
    accelerator.backward(loss)
    #if step % gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1: 把if刪掉了
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    completed_steps += 1

    if step % 50 == 0:
      print({'epoch': epoch, 'step': step, 'loss': loss.item()})

    if completed_steps >= max_train_steps:
      break
      
  logger.info("***** Running eval *****")
  model.eval()
  for step, batch in enumerate(tqdm(eval_dataloader, desc="eval Iteration")):
    outputs = model(**batch)
    predictions = outputs.logits.argmax(dim=-1)
    metric.add_batch(
        predictions=accelerator.gather(predictions),
        references=accelerator.gather(batch["labels"]),
    )

  eval_metric = metric.compute()
  logger.info(f"epoch {epoch}: {eval_metric}")
  if eval_metric['accuracy'] > best_epoch['acc']:
    best_epoch.update({"epoch": epoch, "acc": eval_metric['accuracy']})

  if output_dir is not None:
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir + '/' + 'epoch_' + str(epoch), save_function=accelerator.save)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/306 [00:00<?, ?it/s]

{'epoch': 0, 'step': 0, 'loss': 0.5434992909431458}
{'epoch': 0, 'step': 50, 'loss': 0.40324667096138}
{'epoch': 0, 'step': 100, 'loss': 0.48576590418815613}
{'epoch': 0, 'step': 150, 'loss': 0.4319169223308563}
{'epoch': 0, 'step': 200, 'loss': 0.5567691922187805}
{'epoch': 0, 'step': 250, 'loss': 0.4482481777667999}
{'epoch': 0, 'step': 300, 'loss': 0.48183760046958923}


eval Iteration:   0%|          | 0/34 [00:00<?, ?it/s]

## 最好的某次訓練成果

In [ ]:
print(best_epoch)

{'epoch': 0, 'acc': 0.7769607843137255}


# 訓練成果驗證

In [ ]:
from transformers import BertTokenizerFast, BertConfig, BertForSequenceClassification, default_data_collator
from torch.utils.data import DataLoader
from accelerate import Accelerator
from tqdm.auto import tqdm

In [ ]:
#cd drive

In [ ]:
#ls

In [ ]:
config = BertConfig.from_pretrained("./drive/Shareddrives/epoch_0/config.json") 
model = BertForSequenceClassification.from_pretrained("./drive/Shareddrives/epoch_0/pytorch_model.bin", config = config).to(device)

In [ ]:
from transformers import BertTokenizerFast, BertConfig, BertForSequenceClassification

In [ ]:
def mrpc_model(model, sen1, sen2):
  input_encodings = tokenizer([sen1], [sen2], padding='max_length', truncation=True)
  input_dataset = Dataset(input_encodings)
  #print(input_encodings)
  #print(input_dataset[0])
  data_collator = default_data_collator
  input_dataloader = DataLoader(input_dataset, collate_fn=data_collator, batch_size=1)  

  accelerator = Accelerator()
  model, input_dataloader = accelerator.prepare(model, input_dataloader)

  for batch in input_dataloader:
    outputs = model(**batch)
    predicted = outputs.logits.argmax(dim=-1)
  return predicted

### 可以拿來玩的地方

In [ ]:
sen1="lisa goes to school everyday"
sen2="lisa everyday goes to school"
#sen1="lisa is a singer"
#sen2="lisa is not a singer"

predict = mrpc_model(model, sen1, sen2)
print("sentence= : ", sen1)
print("sentence= : ", sen2)

print("predict_label : ", predict.item())
if predict.item():
  print("有關聯")
else:
  print("沒關聯")


sentence= :  lisa goes to school everyday
sentence= :  lisa everyday goes to school
predict_label :  1
equivalent


In [ ]:
cnt=0
errorcnt=0
#test_sen1,test_sen2,testlabel
for i in range(len(testlabel)):
    cnt+=1
    sen1=test_sen1[i]
    sen2=test_sen2[i]
    predict=mrpc_model(model,sen1,sen2)
    if predict.item()!=testlabel[i]:
        errorcnt+=1
print("cnt = ",cnt)
print("errorcnt = ",errorcnt)

1725
335


結果存進google drive

In [ ]:
"""from google.colab import drive
drive.mount('/content/gdrive')"""

In [ ]:
#cd gdrive/MyDrive/Colab Notebooks

In [ ]:
#torch.save(model,"./test_model2.bin") 